In [2]:
%load_ext nb_black
import sys

sys.path.append("/Users/vikaspandey/my_projects/deeplearning_ai")
import pandas as pd

pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns
import datetime
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from data_visualisation.plot import Plot
from utils.utils import read_csv_in_pd
from utils.utils import get_or_drop_na_cols
from utils.utils import replace_categorical_nan_cols
from utils.utils import replace_numerical_nan_cols
from utils.utils import convert_categorical_to_one_hot_encoding
from utils.utils import get_cols_with_nan_values

<IPython.core.display.Javascript object>

In [3]:
data = read_csv_in_pd("credit-card-fraud-detection", "creditcard.csv")

<IPython.core.display.Javascript object>

In [6]:
data["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

<IPython.core.display.Javascript object>

In [7]:
### Dependent and independent features
X = data.drop("Class", axis=1)
Y = data.Class

<IPython.core.display.Javascript object>

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.7)

<IPython.core.display.Javascript object>

In [17]:
log_class = LogisticRegression()
grid = {"C": 10.0 ** np.arange(-2, 3), "penalty": ["l1", "l2"]}
cv = KFold(n_splits=5, shuffle=False, random_state=None)

<IPython.core.display.Javascript object>

In [19]:
clf = GridSearchCV(log_class, grid, cv=cv, n_jobs=-1, scoring="f1_macro")
clf.fit(X_train, y_train)


/Users/vikaspandey/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'penalty': ['l1', 'l2']},
             scoring='f1_macro')

<IPython.core.display.Javascript object>

In [22]:
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[85260    27]
 [   45   111]]
0.9991573329588147
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85287
           1       0.80      0.71      0.76       156

    accuracy                           1.00     85443
   macro avg       0.90      0.86      0.88     85443
weighted avg       1.00      1.00      1.00     85443



<IPython.core.display.Javascript object>

In [24]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier()

<IPython.core.display.Javascript object>

In [25]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[85282     5]
 [   32   124]]
0.9995669627705019
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85287
           1       0.96      0.79      0.87       156

    accuracy                           1.00     85443
   macro avg       0.98      0.90      0.93     85443
weighted avg       1.00      1.00      1.00     85443



<IPython.core.display.Javascript object>

#### Under Sampling
- Reduce the points of maximum labels

In [30]:
from imblearn.under_sampling import NearMiss
from collections import Counter

ns = NearMiss(0.8)
X_train_ns, y_train_ns = ns.fit_sample(X_train, y_train)
print("Number of classes before fit {}".format(Counter(y_train)))
print("Number of classes after fit {}".format(Counter(y_train_ns)))

/Users/vikaspandey/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/imblearn/utils/_validation.py:635: FutureWarning: Pass sampling_strategy=0.8 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


Number of classes before fit Counter({0: 199028, 1: 336})
Number of classes after fit Counter({0: 420, 1: 336})


<IPython.core.display.Javascript object>

In [32]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(X_train_ns, y_train_ns)

RandomForestClassifier()

<IPython.core.display.Javascript object>

In [33]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[73829 11458]
 [   13   143]]
0.865746755146706
              precision    recall  f1-score   support

           0       1.00      0.87      0.93     85287
           1       0.01      0.92      0.02       156

    accuracy                           0.87     85443
   macro avg       0.51      0.89      0.48     85443
weighted avg       1.00      0.87      0.93     85443



<IPython.core.display.Javascript object>

#### Over Sampling

In [34]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

os = RandomOverSampler(0.5)
X_train_ns, y_train_ns = os.fit_sample(X_train, y_train)
print("Number of classes before fit {}".format(Counter(y_train)))
print("Number of classes after fit {}".format(Counter(y_train_ns)))

/Users/vikaspandey/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/imblearn/utils/_validation.py:635: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


Number of classes before fit Counter({0: 199028, 1: 336})
Number of classes after fit Counter({0: 199028, 1: 99514})


<IPython.core.display.Javascript object>

In [35]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(X_train_ns, y_train_ns)

RandomForestClassifier()

<IPython.core.display.Javascript object>

In [36]:
y_pred = classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[85284     3]
 [   31   125]]
0.999602073897218
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85287
           1       0.98      0.80      0.88       156

    accuracy                           1.00     85443
   macro avg       0.99      0.90      0.94     85443
weighted avg       1.00      1.00      1.00     85443



<IPython.core.display.Javascript object>

In [37]:
from imblearn.combine import SMOTETomek

<IPython.core.display.Javascript object>

In [39]:
os = SMOTETomek(0.5)
X_train_os, y_train_os = os.fit_sample(X_train, y_train)
print("Number of classes before fit {}".format(Counter(y_train)))
print("Number of classes after fit {}".format(Counter(y_train_os)))

/Users/vikaspandey/opt/anaconda3/envs/datascience/lib/python3.8/site-packages/imblearn/utils/_validation.py:635: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.9 "


Number of classes before fit Counter({0: 199028, 1: 336})
Number of classes after fit Counter({0: 198288, 1: 98774})


<IPython.core.display.Javascript object>

#### Ensemble Techniques

In [41]:
from imblearn.ensemble import EasyEnsembleClassifier

<IPython.core.display.Javascript object>

In [42]:
easy = EasyEnsembleClassifier()
easy.fit(X_train, y_train)

EasyEnsembleClassifier()

<IPython.core.display.Javascript object>